In [1]:
pip install pygame

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
    --------------------------------------- 0.3/10.6 MB ? eta -:--:--
   - -------------------------------------- 0.5/10.6 MB 1.5 MB/s eta 0:00:07
   -- ------------------------------------- 0.8/10.6 MB 1.0 MB/s eta 0:00:10
   --- ------------------------------------ 1.0/10.6 MB 1.1 MB/s eta 0:00:09
   ---- ----------------------------------- 1.3/10.6 MB 1.2 MB/s eta 0:00:09
   ------ --------------------------------- 1.8/10.6 MB 1.4 MB/s eta 0:00:07
   ------- -------------------------------- 2.1/10.6 MB 1.5 MB/s eta 0:00:06
   --------- ------------------------------ 2.6/10.6 MB 1.5 MB/s eta 0:00:06
   --------- ------------------------------ 2.6/10.6 MB 1.5 MB/s eta 0:00:06
   ----------- ---------------------------- 3.1/10.6 MB 1.5 MB/s eta 0:00:06
   ------------ ---


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# snake_game.py
import pygame
import random
import sys

# ---------------- SETTINGS ----------------
CELL_SIZE = 20
GRID_WIDTH, GRID_HEIGHT = 30, 20
SCREEN_WIDTH, SCREEN_HEIGHT = GRID_WIDTH * CELL_SIZE, GRID_HEIGHT * CELL_SIZE
FPS = 10

# Colors
WHITE = (255, 255, 255)
GREEN = (34, 139, 34)
RED = (220, 20, 60)
BLACK = (0, 0, 0)
GRAY = (40, 40, 40)

# Directions
UP, DOWN, LEFT, RIGHT = (0, -1), (0, 1), (-1, 0), (1, 0)


class Snake:
    def __init__(self):
        self.body = [(GRID_WIDTH // 2, GRID_HEIGHT // 2)]
        self.direction = RIGHT
        self.grow = False

    def move(self):
        head_x, head_y = self.body[0]
        dx, dy = self.direction
        new_head = (head_x + dx, head_y + dy)

        # Insert new head
        self.body.insert(0, new_head)

        if self.grow:
            self.grow = False
        else:
            self.body.pop()

    def change_direction(self, new_dir):
        # Prevent 180° turns
        opposite = (-self.direction[0], -self.direction[1])
        if new_dir != opposite:
            self.direction = new_dir

    def collides(self):
        head = self.body[0]
        # Wall collision
        if not (0 <= head[0] < GRID_WIDTH and 0 <= head[1] < GRID_HEIGHT):
            return True
        # Self collision
        if head in self.body[1:]:
            return True
        return False

    def eat(self, food_pos):
        if self.body[0] == food_pos:
            self.grow = True
            return True
        return False


class Food:
    def __init__(self, snake_body):
        self.position = self.random_pos(snake_body)

    def random_pos(self, snake_body):
        while True:
            pos = (random.randint(0, GRID_WIDTH - 1),
                   random.randint(0, GRID_HEIGHT - 1))
            if pos not in snake_body:
                return pos


class Game:
    def __init__(self):
        pygame.init()
        self.screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
        pygame.display.set_caption("Snake Game")
        self.clock = pygame.time.Clock()
        self.font = pygame.font.SysFont("Arial", 24, bold=True)
        self.reset()

    def reset(self):
        self.snake = Snake()
        self.food = Food(self.snake.body)
        self.score = 0

    def draw_grid(self):
        for x in range(0, SCREEN_WIDTH, CELL_SIZE):
            pygame.draw.line(self.screen, GRAY, (x, 0), (x, SCREEN_HEIGHT))
        for y in range(0, SCREEN_HEIGHT, CELL_SIZE):
            pygame.draw.line(self.screen, GRAY, (0, y), (SCREEN_WIDTH, y))

    def draw_snake(self):
        for x, y in self.snake.body:
            rect = pygame.Rect(x * CELL_SIZE, y * CELL_SIZE, CELL_SIZE, CELL_SIZE)
            pygame.draw.rect(self.screen, GREEN, rect)

    def draw_food(self):
        x, y = self.food.position
        rect = pygame.Rect(x * CELL_SIZE, y * CELL_SIZE, CELL_SIZE, CELL_SIZE)
        pygame.draw.rect(self.screen, RED, rect)

    def draw_score(self):
        text = self.font.render(f"Score: {self.score}", True, WHITE)
        self.screen.blit(text, (10, 10))

    def game_over(self):
        text = self.font.render("GAME OVER - Press R to Restart", True, RED)
        self.screen.blit(text, (SCREEN_WIDTH//6, SCREEN_HEIGHT//2))
        pygame.display.flip()

        waiting = True
        while waiting:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_r:
                        self.reset()
                        waiting = False

    def run(self):
        while True:
            # --- Events ---
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_UP:
                        self.snake.change_direction(UP)
                    elif event.key == pygame.K_DOWN:
                        self.snake.change_direction(DOWN)
                    elif event.key == pygame.K_LEFT:
                        self.snake.change_direction(LEFT)
                    elif event.key == pygame.K_RIGHT:
                        self.snake.change_direction(RIGHT)

            # --- Update ---
            self.snake.move()
            if self.snake.eat(self.food.position):
                self.score += 1
                self.food = Food(self.snake.body)

            if self.snake.collides():
                self.game_over()

            # --- Draw ---
            self.screen.fill(BLACK)
            self.draw_grid()
            self.draw_snake()
            self.draw_food()
            self.draw_score()

            pygame.display.flip()
            self.clock.tick(FPS)


if __name__ == "__main__":
    Game().run()


pygame 2.6.1 (SDL 2.28.4, Python 3.13.5)
Hello from the pygame community. https://www.pygame.org/contribute.html
